
## cn_stock/bars/daily

### Overwrite

In [3]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    with open("finance.duckdb.sql") as f:
        # conn.execute("drop table bars_daily_000001_SZ;")
        sql = f.read().format(tablename='000001_SZ', csv_path='datasources/000001.SZ')
        conn.execute(sql)
    conn.table('bars_daily_000001_SZ').show()

┌─────────────────────┬──────────┬───────┬───────┬───────┬───┬───────────────┬─────────┬─────────┬─────────┬─────────┐
│         dt          │   name   │ _open │ _high │ _low  │ … │ lb_down_count │  close  │  open   │  high   │   low   │
│     timestamp_s     │ varchar  │ float │ float │ float │   │     uint8     │  float  │  float  │  float  │  float  │
├─────────────────────┼──────────┼───────┼───────┼───────┼───┼───────────────┼─────────┼─────────┼─────────┼─────────┤
│ 1991-01-03 00:00:00 │ 深发展Ａ │  66.4 │  66.4 │  66.4 │ … │             0 │ 7.07077 │ 7.07077 │ 7.07077 │ 7.07077 │
│ 1991-01-04 00:00:00 │ 深发展Ａ │ 66.07 │ 66.07 │ 66.07 │ … │             0 │ 7.03562 │ 7.03562 │ 7.03562 │ 7.03562 │
│ 1991-01-07 00:00:00 │ 深发展Ａ │ 66.42 │ 66.42 │ 66.42 │ … │             0 │  7.0729 │  7.0729 │  7.0729 │  7.0729 │
│ 1991-01-09 00:00:00 │ 深发展Ａ │ 65.76 │ 65.76 │ 65.76 │ … │             0 │ 7.00261 │ 7.00261 │ 7.00261 │ 7.00261 │
│ 1991-01-10 00:00:00 │ 深发展Ａ │ 65.43 │ 65.43 │ 65.43 │ … │      

### Write

In [4]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    insert_sql = f"INSERT INTO bars_daily_000001_SZ SELECT dt,name,_open,_high,_low,_close,volume,amount,preclose,net_profit_ttm,cashflow_ttm,equity,asset,debt,debttoasset,net_profit_q,pe_ttm,pb,mkt_cap,mkt_cap_ashare,vip_buy_amt,vip_sell_amt,inst_buy_amt,inst_sell_amt,mid_buy_amt,mid_sell_amt,indi_buy_amt,indi_sell_amt,master_net_flow_in,master2_net_flow_in,vip_net_flow_in,mid_net_flow_in,inst_net_flow_in,indi_net_flow_in,total_sell_amt,total_buy_amt,net_flow_in,turnover,free_shares,total_shares,maxupordown,maxupordown_at_open,lb_up_count,lb_down_count,close,open,high,low FROM 'datasources/data_append.csv' where symbol='000001.SZ';"
    conn.execute(insert_sql)

### Delete

In [5]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    # 删除不存在的行不会报错
    insert_sql = f"DELETE FROM bars_daily_000001_SZ WHERE dt = '2024-09-02 00:00:00';"
    conn.execute(insert_sql)

### Read

In [6]:
import quantdata as qd

with qd.duckdb_connect('datas/duckdb/test_finance.db') as conn:
    # df = qd.duckdb_get_array(conn, 'bars_daily_000001_SZ').df()
    # df = qd.duckdb_get_array(conn, 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # df = qd.duckdb_get_array(conn, 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").df()
    # 最后一行(所有列)
    # df = qd.duckdb_get_array_last_rows(conn, 'bars_daily_000001_sz').df()
    # 最后一行(自定义列)
    # df = qd.duckdb_get_array_last_rows(conn, 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # 最后5行
    df = qd.duckdb_get_array_last_rows(conn, 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], N=5).df()
    # df = qd.duckdb_get_array_last_rows(conn, 'bars_daily_000001_SZ', N=1).df()
df

,dt,name,open
0,2024-08-26,平安银行,10.50
1,2024-08-27,平安银行,10.47
2,2024-08-28,平安银行,10.41
3,2024-08-29,平安银行,10.31
4,2024-08-30,平安银行,10.11


### Benchmark

以下为在大数据(全部5000+只股票的数据，一共5G)测试性能

In [1]:
import quantdata as qd

with qd.duckdb_connect('datas/duckdb/finance.db') as conn:
    %timeit qd.duckdb_get_array(conn, 'bars_daily_000001_SZ').fetchnumpy()
    %timeit  qd.duckdb_get_array(conn, 'bars_daily_000002_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    %timeit  qd.duckdb_get_array(conn, 'bars_daily_000003_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").fetchnumpy()
    # # 最后一行(所有列)
    %timeit  qd.duckdb_get_array_last_rows(conn, 'bars_daily_000005_SZ').fetchnumpy()
    # # 最后一行(自定义列)
    %timeit  qd.duckdb_get_array_last_rows(conn, 'bars_daily_000008_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    # # 最后5行
    %timeit qd.duckdb_get_array_last_rows(conn, 'bars_daily_000010_SZ', attrs=['dt', 'name', 'open'], N=5).fetchnumpy()
    %timeit  qd.duckdb_get_array_last_rows(conn, 'bars_daily_000001_SZ', N=5).fetchnumpy()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

5.76 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.23 ms ± 33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
669 µs ± 8.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.23 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
983 µs ± 43.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
967 µs ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.46 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1]:
import duckdb

def duckdb_connect(uri):
    # return duckdb.execute(f"ATTACH '{uri}' as A (READ_ONLY);USE A;")
    conn = duckdb.connect(config = {'threads': 12})
    # conn = duckdb.connect(config = {'threads': 12, 'enable_progress_bar':False, 'enable_progress_bar_print': False})
    try:
        conn.execute(f"ATTACH '{uri}' as A (READ_ONLY);USE A;")
        return conn
    except:
        conn.close()
        raise

def duckdb_connect2(uri):
    conn = duckdb.connect(uri, read_only=True)
    try:
        # conn.execute(f"ATTACH '{uri}' as A (READ_ONLY);USE A;")
        return conn
    except:
        conn.close()
        raise


def duckdb_close(conn):
    conn.close()


def test_conn():
    # conn = duckdb_connect('datas/duckdb/finance.db')
    conn = duckdb_connect('//DESKTOP-6NI30V4/bt_data/duckdb/finance.db')
    duckdb_close(conn)

test_conn()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))